In [ ]:
# if you want to use wandb for visualization you can install it in your workbook as show below
!pip install wandb -qU

In [ ]:
# Log in to your W&B account
import wandb
wandb.login(key='') # add your key here to log in to wandb if you want to use it for visualization

In [ ]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import csv
import gc
import math
from torchsummary import summary
from datetime import datetime
import random

In [ ]:
#inputs
# Size of z latent vector (batch_idx.e. size of generator input)
nz = 512

# Size of window for appliance samples, batch size and max epochs declaration
window_size = 1500 #4319  #21599 # 6 hour windows
batch_size = 8
max_epochs = 200

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Parameters
params = {'batch_size': batch_size,
          'shuffle': True,
            'num_workers': 2,
            'drop_last': True} # using drop_last = True so that samples returned are divisible by batch size


In [ ]:
# our custom dataloader used to load the data into the GAN architecture
class Dataset(torch.utils.data.Dataset):
    

    def __init__(self, df, window_size):
        'Initialization'
        # convert to numpy and create tensors from the data
        x = df.iloc[:, :].to_numpy()
        self.window_size = window_size

        self.x_train = torch.tensor(list(x))

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.x_train)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        return self.x_train[index].reshape(1, self.window_size)

The Generator and Discriminator architectures


In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()

    def forward(self, x):
        # Print for debugging purposes
        print(x)
        print(x.size())
        return x

class Generator(nn.Module):

    def __init__(self, input, in_channels, out_channels, kernel_size,kernel_size2, output, groups=1):
        super(Generator, self).__init__()

        left, right = kernel_size // 2, kernel_size // 2
        if kernel_size % 2 == 0:
            right -= 1
        padding = (left, right, 0, 0)

        interm = input-2*(kernel_size -1) - 1*(kernel_size2 -1)
        interm_2 = (interm - 1)*2 + kernel_size
        interm_3 = (interm_2 -1)*2 + kernel_size
        interm_4 = (interm_3 -1)*2 + kernel_size

        self.conv = nn.Sequential(
            nn.Conv1d(in_channels, 16, kernel_size, groups=groups),
            nn.LeakyReLU(0.1, inplace=False),
            nn.Dropout(p=0.1),
            nn.Conv1d(16, 16, kernel_size2, groups=groups),
            nn.LeakyReLU(0.1, inplace=False),
            nn.Conv1d(16, 4, kernel_size, groups=groups),
            nn.BatchNorm1d(4),
            nn.LeakyReLU(0.1, inplace=False),
        )

        self.deconv = nn.Sequential(
            nn.ConvTranspose1d(4, 2, kernel_size, stride=2, groups=groups),
            nn.LeakyReLU(0.1, inplace=False),
            nn.ConvTranspose1d(2, 2, kernel_size, stride=2, groups=groups),
            nn.LeakyReLU(0.1, inplace=False),
            nn.ConvTranspose1d(2, 1, kernel_size, stride=2, groups=groups),
            nn.LeakyReLU(0.1, inplace=False),
        )

        self.linear = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(interm_4, interm_4*2,bias=True),
            nn.BatchNorm1d(interm_4*2), #for more stability
            nn.LeakyReLU(0.1, inplace=True),
        )

        self.outlayer = nn.Sequential(

           nn.Linear( interm_4*2, output,bias=True),

        )

    def forward(self, x):
        x = self.conv(x)
        x = self.deconv(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = self.outlayer(x)
        x = nn.Tanh()(x)

        return x


# Discriminator Code
class Discriminator(nn.Module):
    def __init__(self, input, in_channels, kernel_size, groups ):
        super(Discriminator, self).__init__()

        left, right = kernel_size // 2, kernel_size // 2
        if kernel_size % 2 == 0:
            right -= 1
        padding = (left, right, 0, 0)

        interm = input-(kernel_size-1)-1*(9-1)
        interm_2 =math.ceil((interm - 14)/2)
        interm_4 = math.ceil((interm_2 - 8)/2)

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels=16, kernel_size=kernel_size, groups=groups, padding=0),
            nn.LeakyReLU(0.1, inplace=False),
            nn.Dropout(p=0.1),
            nn.Conv1d(16, 8, kernel_size=9, groups=groups, padding=0),
            nn.LeakyReLU(0.1, inplace=False),
        )

        self.conv = nn.Sequential(

            nn.Conv1d(8, 3, kernel_size=15,stride=2, groups=groups, padding=0),
            nn.LeakyReLU(0.1, inplace=False),
            nn.Dropout(p=0.1),
            nn.Conv1d(3, 1, kernel_size=9, stride=2, groups=groups, padding=0),
            nn.LeakyReLU(0.1, inplace=False)

        )

        self.linear = nn.Sequential(
            nn.Linear(interm_4 , round(interm_4/2),bias=True),  #fully connected layer
            nn.LeakyReLU(0.05, inplace=True),
            nn.Linear(round(interm_4/2), 1,bias=True),  #fully connected layer
            nn.Sigmoid()

        )

    def forward(self, x):
        x = self.main(x)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)

        return x

In [ ]:
# custom weights initialization called on netG and netD
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)



# create latent noise from gaussian distribution
def noise_samples(batch_size, nz ):
    fixed_noise = (torch.randn(batch_size, 1, nz, device=device) - 0.5) / 0.5
    #fixed_noise = torch.rand(batch_size, 1, nz, device=device)
    #fixed_noise = torch.normal(0.5, 1, size=(batch_size, 1, nz), device=device)
    #fixed_noise = torch.zeros(batch_size, 1, nz, device=device)
    return fixed_noise

In [ ]:
# read the final and cleaned dataset for the appliance
train_pre = pd.read_pickle('path_to_saved/dataport_electric_vehicle_5s_clean.pkl')

In [ ]:
# load the data into the dataloader
training_set = Dataset(train_pre, window_size)
training_dataloader = torch.utils.data.DataLoader(training_set, **params)

In [ ]:
#initialize the discriminator network
netD = Discriminator(input=window_size, in_channels=1, kernel_size=25, groups=1).to(device)
if torch.cuda.is_available():
  netD.cuda()
netD.apply(weights_init)
print(netD)

# initialize the generator network
netG = Generator(input= nz, in_channels=1, out_channels=1, kernel_size=25, kernel_size2=75, output=window_size).to(device)
if torch.cuda.is_available():
  netG.cuda()

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)
# Print the model
print(netG)

In [ ]:
SEED=42 #42
torch.manual_seed(SEED)

The main train loop!


In [ ]:
if __name__ == '__main__':

  torch.cuda.empty_cache()
  adversarial_loss = torch.nn.BCELoss()
  # Initialize the loss function
  kl_loss = nn.KLDivLoss(reduction="batchmean") 
  criterion = nn.MSELoss()
  softmax = torch.nn.Softmax()
  fixed_noise = noise_samples(batch_size, nz )

  # Establish convention for real and fake labels during training - label smoothing
  real_label = 0.9
  fake_label = 0  # could also try 0.1

  # Learning rate for optimizers
  lr_d = 0.00004
  lr_g = 0.00007

  now = datetime.now()

  # if you use wandb, the below code can create dashboards for review of the experiments, if not, 
  # you should delete this
  wandb.init(
    # Set the project where this run will be logged
    project="ddgan-nilm",
    name=f"experiment_1 "+ now.strftime("%m/%d/%Y, %H:%M:%S"),   # add timestamp to name 
    # Track hyperparameters and run metadata
    config={
    "lr_d": lr_d,
    "lr_g": lr_g,
    "batch_size":batch_size,
    "window_size":window_size,
    "nz": nz,
    "loss": criterion,
    "architecture": "DDGAN",
    "dataset": "1s_data_newyork_file",
    "epochs": max_epochs,
    })

  # Setup Adam optimizers for both G and D
  optimizerD = optim.Adam(netD.parameters(), lr=lr_d, betas=(0.5, 0.999))
  optimizerG = optim.Adam(netG.parameters(), lr=lr_g, betas=(0.5, 0.999))


  energ_list = []
  G_losses = []
  D_losses = []
  iters = 0

  for epoch in range(max_epochs):
      # Training
      # save the model every couple of epochs in case training fails or is stopped
      print('epoch: ', epoch)
      if (epoch % 10 == 0):
              torch.save(netD, 'path_to_saved/discriminator_temp_model.pt')
              torch.save(netG, 'path_to_saved/generator_temp_model.pt')
              print('saved models!')
      for batch_idx, data in enumerate(training_dataloader, 0):

          #print('batch index: ',batch_idx)
          #print(data)

          ############################
          # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
          ###########################

          # Train with all-real batch
          optimizerD.zero_grad()

          # Format batch
          real_cpu = data.to(device)
          #print(real_cpu)
          #print(real_cpu.shape)

          b_size = real_cpu.size(0)
          #print('b size: ', b_size)

          #create the real labels for the batch
          label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
          #print('labels: ', label)
          #print(label.shape)
          #print(real_cpu)
          #print(real_cpu.shape)

          # Forward pass real batch through D
          output = netD(real_cpu).view(-1)
          #print('output ',output)
          # Calculate loss on full real batch
          #print('real output:')
          #print(output)
          #print('labels:')
          #print(label)

          errD_real = adversarial_loss(output, label)

          # Calculate gradients for D in backward pass
          errD_real.backward()
          D_x = output.mean().item()

          # Train with all-fake batch
          # Generate batch of latent vectors
          noise = noise_samples(batch_size, nz )

          # Generate fake time series batch with G
          fake = netG(noise)
          #print('fake ', fake)
          label.fill_(fake_label)
          # Classify all fake batch with D
          #print(fake)
          fake = fake.reshape(batch_size,1, window_size)
          output = netD(fake.detach()).view(-1)
          #print('fake output:')
          #print(output)
          #print('fake labels:')
          #print(label)
          # Calculate D's loss on the all-fake batch
          errD_fake = adversarial_loss(output, label)
          # Calculate the gradients for this batch, accumulated (summed) with previous gradients
          errD_fake.backward()
          D_G_z1 = output.mean().item()
          # Compute error of D as sum over the fake and the real batches
          errD = errD_real + errD_fake
          # Update D
          optimizerD.step()

          ############################
          # (2) Update G network: maximize log(D(G(z)))
          ###########################
          optimizerG.zero_grad()
          label.fill_(real_label)  # fake labels are real for generator cost
          # Since we just updated D, perform another forward pass of all-fake batch through D
          output = netD(fake).view(-1)
          # Calculate G's loss based on this output
          errG = criterion(output, label)
          # Calculate gradients for G
          errG.backward()
          D_G_z2 = output.mean().item()
          # Update G
          optimizerG.step()

          # Output training stats
          if batch_idx % 50 == 0:
              print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                      % (epoch, max_epochs, batch_idx, len(training_dataloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

          # Save Losses for plotting later
          G_losses.append(errG.item())
          D_losses.append(errD.item())


          # Check how the generator is doing by saving G's output on fixed_noise
          if (iters % 50 == 0) or ((epoch == max_epochs-1) and (batch_idx == len(training_dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
                fig, axs = plt.subplots(2)
                fig.suptitle('Current fake waves: ')
                axs[0].plot(fake[0])
                axs[1].plot(fake[4])
                plt.show()


            # log some values in wandb
            wandb.log({"Loss_D": errD.item(), "Loss_G": errG.item(), "D(x)": D_x, "D(G(z))_coef1": D_G_z1, "D(G(z))_coef2": D_G_z2})

            energ_list.append(fake)


          iters += 1

  plt.clf()
  plt.title("Generator and Discriminator Loss During Training")
  plt.plot(G_losses,label="G")
  plt.plot(D_losses,label="D")
  plt.xlabel("iterations")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()


  # Mark the run as finished
  wandb.finish()

Save the models that were trained

In [ ]:
torch.save(netD, 'path_to_saved/discriminator_sgan_signature.pt')
torch.save(netG, 'path_to_saved/generator_sgan_signature.pt')

In [ ]:
Gen = torch.load('path_to_saved/generator_sgan_signature.pt')
Gen.eval()

Create final generator results plots

In [ ]:
# create latent vectors to get results
generated_samples = []
noise_test = noise_samples(batch_size, nz )
with torch.no_grad():
    fake = Gen(noise_test).detach().cpu()
generated_samples.append(fake)

fig, axs = plt.subplots(4)
fig.suptitle('Multiple fake and real waves')
axs[0].plot(generated_samples[0][0].reshape(-1))
axs[1].plot(generated_samples[0][2].reshape(-1))

real_batch = next(iter(training_dataloader))
real = real_batch.reshape(batch_size, window_size, 1)
axs[2].plot(real[0])
axs[3].plot(real[1])

plt.show()


Read the saved dataset with the 6 hour windows, before the cleaning, to calculate the start times of the devices


In [ ]:
train_pre = pd.read_pickle('path_to_saved/dataport_electric_vehicle_5s.pkl')
print(train_pre)

In [ ]:
indices = []
for index, data in train_pre.iterrows():
  indices.append(next((i for i, x in enumerate(data[0]) if x>0.1), None))

Create the final outputs with post process


In [ ]:
# set the size of the final window to be created
final_window = 4319 

final = []
for fake in generated_samples[0]:
  place =random.choice(indices)
  result = torch.zeros(1, place)
  result = torch.reshape(result, (-1,))
  fake[fake < 0] = 0
  test = torch.reshape(fake, (-1,))
  result = torch.cat((result,test ), 0)
  rest  = final_window-window_size-place if final_window-window_size-place>0 else 0
  end = torch.zeros(1, rest)
  end = torch.reshape(end, (-1,))
  fake_final  =  torch.cat((result,end ), 0)
  fake_final = fake_final[:final_window]
  final.append(fake_final)

final = torch.stack(final)

# if the size exceeds the max trim the window
fig, axs = plt.subplots(8)
fig.suptitle('Multiple fake and real waves')
axs[0].plot(final[0])
axs[1].plot(final[1])
axs[2].plot(final[2])
axs[3].plot(final[3])
axs[4].plot(final[4])
axs[5].plot(final[5])
axs[6].plot(final[6])
axs[7].plot(final[7])
plt.show()